### ¿Qué hace este script?

#### Calcula para cada AP: 
* usuarios.consumoUsuariosDescarga
* usuarios.consumoUsuariosCarga
* usuarios.consumoUsuarios, que es el resultado de la suma de los dos anteriores

In [2]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [3]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
)

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [4]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### nombre de indice donde se insertará

In [5]:
indice = parametros.mintic_concat_index
indice_control = parametros.mintic_control

### Función para generar JSON compatible con ES

In [6]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [7]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["usuarios.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_usuarios_consumos"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['usuarios.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-05-01 00:00:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:
* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISO, id_Beneficiario, entre otros.

In [8]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud', 'COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

### Cambiando nombre de campos y generando location

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [9]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['usuarios.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['usuarios.location']=datos_semilla['usuarios.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

In [10]:
datos_semilla = datos_semilla.rename(columns={'lugar_cod' : 'usuarios.centroDigitalUsuarios'
                                            , 'nombre_municipio': 'usuarios.nombreMunicipio'
                                            , 'nombre_departamento' : 'usuarios.nombreDepartamento'
                                            , 'nombre_centro_pob': 'usuarios.localidad'
                                            , 'nombreSede':'usuarios.nomCentroDigital'
                                            , 'energiadesc' : 'usuarios.sistemaEnergia'
                                            , 'COD_ISO' : 'usuarios.codISO'
                                            , 'id_Beneficiario' : 'usuarios.idBeneficiario'})

Se descartan los registros que tengan la latitud y longitud vacía o no valida

In [11]:
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["usuarios.location"]=='')].index)

### leyendo indice cambium-devicedevices

De esta formas se asocia las MAC de dispositivos de red INDOOR y OUTDOOR
* site_id para cruzar con las misma llave de semilla.
* datos del dispositivo: mac, status, ip.
* ap_group para identificar si la conexión es indoor/outdoor

In [12]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","status","ip","ap_group"]
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

Se descartan registros con site_id vacios ya que no cruzarán en el merge y se limpian los NaN del dataframe.

In [13]:
datos_dev.dropna(subset=['site_id'], inplace=True)
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)

Se corrigen datos de ap group con formato no valido

In [14]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

Se toman solo los datos unicos con site_id y mac.

In [15]:
#datos_dev.drop_duplicates(subset=['site_id','mac'],inplace=True)
datos_dev = datos_dev.drop_duplicates('mac')

Se cambia el nombre a la mac del dispositivo de red para no confundir con la de dispositivos de usuario 

In [16]:
datos_dev= datos_dev.rename(columns={'mac' : 'usuarios.macRed','ap_group' : 'usuarios.apGroup'})

## Se lee la información de cambium device client

 Se toma los valores de dispositivos y su desempeño. De esta lectura se toma el consumo de carga y descarga para cada dispositivo de usuario en una fecha dada. 
 * mac corresponde al dispositivo de usuario.
 * ap_mac corresponde al dispositivo de red.
 * timestamp es la fecha y hora de la medición
 * radio.* volumen de datos descargados(r) y cargados(t)


In [18]:
def traeRadio(fecha_max,fecha_tope):
    ttotal_docs = 100000
    response = es.search(
        index= 'all-'+parametros.cambium_d_c_index,
        body={
            "_source": ['mac', 'ap_mac', 'radio.band', 'radio.rx_bytes', 'radio.tx_bytes'
                  ,'fecha_control']
              , "query": {
                  "range": {
                    "fecha_control": {
                      "gte": fecha_max,
                      "lt": fecha_tope
                      #"gte": "2021-05-26 15:00:00",
                      #"lt": "2021-05-26 15:10:00"  
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

### Se ejecuta consulta

* Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [19]:
fecha_max_mintic = fecha_ejecucion

fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_performance = traeRadio(fecha_max_mintic,fecha_tope_mintic)

if datos_performance is None or datos_performance.empty:
    while (datos_performance is None or datos_performance.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        datos_performance = traeRadio(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

* Se calcula fecha control.
* Se toma la maxima medición de radios(rx y tx) para cada AP y banda.
* Se suman los radios agrupando por AP y fecha control
* Se convierten a Gb los resultados

In [20]:
try:  
    datos_performance['fecha_control'] = datos_performance["fecha_control"].str[0:-4] + '0:00'
    datos_performance.replace('','0',inplace=True)
    datos_performance.fillna({'radio.rx_bytes':0,'radio.tx_bytes':0},inplace=True)
    datos_performance['radio.rx_bytes'] = datos_performance['radio.rx_bytes'].astype(float)
    datos_performance['radio.tx_bytes'] = datos_performance['radio.tx_bytes'].astype(float)
    
    datos_performance = datos_performance.groupby(['ap_mac', 'fecha_control', 'mac', 'radio.band']).agg(['max']).reset_index()
    datos_performance.columns = datos_performance.columns.droplevel(1)
    datos_performance = datos_performance[['ap_mac'
                                         , 'fecha_control'
                                         , 'radio.rx_bytes'
                                         , 'radio.tx_bytes']].groupby(['ap_mac'
                                                                     , 'fecha_control']).agg(['sum']).reset_index()
    datos_performance.columns = datos_performance.columns.droplevel(1)
    datos_performance = datos_performance.rename(columns={'radio.rx_bytes': 'usuarios.consumoUsuariosDescarga_aux'
                                                     , 'radio.tx_bytes':'usuarios.consumoUsuariosCarga_aux'})    
    
    
    datos_performance['usuarios.consumoUsuariosDescarga'] = round((datos_performance['usuarios.consumoUsuariosDescarga_aux']/float(1<<30)),6)
    datos_performance['usuarios.consumoUsuariosCarga'] = round((datos_performance['usuarios.consumoUsuariosCarga_aux']/float(1<<30)),6)
    datos_performance['usuarios.consumoUsuarios'] = datos_performance['usuarios.consumoUsuariosDescarga'] + datos_performance['usuarios.consumoUsuariosCarga']
    datos_performance['usuarios.consumoUsuarios'] = round(datos_performance['usuarios.consumoUsuarios'],6)
    datos_performance = datos_performance.rename(columns={'ap_mac':'usuarios.macRed'})
    
    datos_performance = pd.merge(datos_performance,datos_dev[['site_id','usuarios.macRed','usuarios.apGroup']],on='usuarios.macRed', how='left')
    
    aux_performance = pd.merge(datos_performance,  datos_semilla, on='site_id',how='inner')
    aux_performance.fillna({'usuarios.consumoUsuariosDescarga':0,
                              'usuarios.consumoUsuariosCarga':0,
                              'usuarios.consumoUsuarios':0,
                              'usuarios.apGroup':'No identificado'},inplace=True)
    aux_performance = aux_performance.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})
    
    aux_performance["usuarios.fecha"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    aux_performance["usuarios.anyo"] = aux_performance["usuarios.fecha"].str[0:4]
    aux_performance["usuarios.mes"] = aux_performance["usuarios.fecha"].str[5:7]
    aux_performance["usuarios.dia"] = aux_performance["usuarios.fecha"].str[8:10]
    aux_performance["usuarios.hora"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    aux_performance["usuarios.minuto"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    aux_performance['nombreDepartamento'] = aux_performance['usuarios.nombreDepartamento']
    aux_performance['nombreMunicipio'] = aux_performance['usuarios.nombreMunicipio']
    aux_performance['idBeneficiario'] = aux_performance['usuarios.idBeneficiario']
    aux_performance['fecha'] = aux_performance['usuarios.fecha']
    aux_performance['anyo'] = aux_performance['usuarios.anyo']
    aux_performance['mes'] = aux_performance['usuarios.mes']
    aux_performance['dia'] = aux_performance['usuarios.dia']
except:
    pass

# Insertando consumo usuarios  al indice principal

In [21]:
try:
    use_these_keys = ['usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.idBeneficiario'    
                  , 'usuarios.localidad'
                  , 'usuarios.siteID'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.location'
                  , 'usuarios.fechaControl'
                  , 'usuarios.macRed'
                  , 'usuarios.apGroup'    
                  , 'usuarios.consumoUsuariosDescarga'
                  , 'usuarios.consumoUsuariosCarga'
                  , 'usuarios.consumoUsuarios'
                  , 'usuarios.fecha'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.hora'
                  , 'usuarios.minuto'
                    , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']

    aux_performance['@timestamp'] = now.isoformat()
    def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{ 'usuariosConsumo-'+ str(document['usuarios.siteID']) + str(document['usuarios.macRed']) + '-' + str(document['usuarios.fechaControl'])}",
                    "_source": filterKeys(document),
                }
    salida = helpers.bulk(es, doc_generator(aux_performance))
    print("Fecha: ", now,"-Consumos de usuarios insertados en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- Ningun consumo usuario para insertar en indice principal")

Fecha:  2021-06-10 16:25:19.094536 -Consumos de usuarios insertados en indice principal: 60


### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [19]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia_usuarios_consumos',
        body = { 'jerarquia_usuarios_consumos': 'usuarios_consumos','usuarios.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-05-26 10:50:00
